# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [22]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [23]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [24]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check the first 2 examples
print('\n',full_data_rows_list[0],'\n', full_data_rows_list[1])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056

 ['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'] 
 ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88']


In [25]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

### Creating a Cluster and Keyspace

In [26]:
#==================
# Create cluster
#==================

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()


#==================
# Create Keyspace
#==================

session.execute("""CREATE KEYSPACE IF NOT EXISTS p2_udacity 
                   WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}""")


#==================
# Set Keyspace
#==================

session.set_keyspace('p2_udacity')

# Create queries to ask the following three questions of the data

> 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


> 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

> 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [27]:
# CSV file will be used in 3 queries
file = 'event_datafile_new.csv'

## Solution Query 1

In [28]:
#====================================================
# Create table
# For table song_features, the itemInSession was 
# used as a partition key because the queries will 
# filter by this column. The sessionId were used as 
# clustering column to help make up a unique key.
#====================================================

# in case rerun this cell, drop the current table
session.execute("DROP TABLE IF EXISTS song_features")

# create table for Query 1
song_features = """CREATE TABLE IF NOT EXISTS song_features (itemInSession int, 
                                sessionId int, artist text, song text,
                                length float, PRIMARY KEY(itemInSession, sessionId) )"""


# execute the create table
session.execute(song_features)

#==================
# Insert the data
#==================

# insert data for song_features
insert_data_song_features = """INSERT INTO song_features (
                         itemInSession, sessionId, 
                         artist, song, length) VALUES (%s, %s, %s, %s, %s)"""


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # assing csv columns in each variable
        itemInSession, sessionId  = int(line[3]), int(line[8])
        artist, song, length = str(line[0]), str(line[9]), float(line[5])
        
        # execute the insertion
        session.execute(insert_data_song_features, (itemInSession, sessionId, artist, song, length))
        
#==================
# Validate the data
#==================



query_1 = """SELECT artist, song, length 
                    FROM song_features 
                    WHERE itemInSession = %s AND sessionId = %s"""

rows = session.execute(query_1, (4, 338))
for row in rows:
    print(f'artist: {row.artist}, song: {row.song} length: {row.length:.8}')        

artist: Faithless, song: Music Matters (Mark Knight Dub) length: 495.30731


## Solution Query 2

In [29]:
#====================================================
# Create table
# For table artist_song_by_user, the userId and 
# sessionId was used as a composite partition key 
# because the queries will filter by these columns. 
# The itemInSession were used as clustering column 
# to help make up a unique key.
#====================================================

# in case rerun this cell, drop the current table 
session.execute("DROP TABLE IF EXISTS artist_song_by_user")

# create table for Query 1
artist_song_by_user = """CREATE TABLE IF NOT EXISTS artist_song_by_user (userId int, sessionId int, itemInSession int, 
                                         artist text, song text, firstName text, 
                                         lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"""
# execute the create table
session.execute(artist_song_by_user)

#================== 
# Insert the data
#==================

insert_data_artist_song_by_user = """INSERT INTO artist_song_by_user (
                         userId, sessionId, itemInSession, 
                         artist, song, firstName, lastName) 
                         VALUES (%s, %s, %s, %s, %s, %s, %s)"""


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # assing csv columns in each variable
        userId, sessionId, itemInSession  = int(line[10]), int(line[8]), int(line[3])
        artist, song, firstName, lastName = str(line[0]), str(line[9]), str(line[1]), str(line[4])
        
        # building the insert
        # execute the insertion
        session.execute(insert_data_artist_song_by_user, (userId, sessionId, itemInSession, artist, song, firstName, lastName))
        
#==================
# Validate the data
#==================

# Query 2 in CQL
query_2 = """SELECT artist, song, firstName, lastName 
                    FROM artist_song_by_user
                    WHERE userId = %s AND sessionId = %s"""

rows = session.execute(query_2, (10, 182))
for row in rows:
    print(f'artist: {row.artist}, song: {row.song}, user first name: {row.firstname}, user last name: {row.lastname}')        

artist: Down To The Bone, song: Keep On Keepin' On, user first name: Sylvie, user last name: Cruz
artist: Three Drives, song: Greece 2000, user first name: Sylvie, user last name: Cruz
artist: Sebastien Tellier, song: Kilometer, user first name: Sylvie, user last name: Cruz
artist: Lonnie Gordon, song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), user first name: Sylvie, user last name: Cruz


## Solution Query 3

In [20]:
#====================================================
# Create table
# For table user_name, the song was used as a 
# partition key because the queries will filter by 
# this column. The userId were used as clustering 
# column to help make up a unique key.
#====================================================

# in case rerun this cell, drop the current table 
session.execute("DROP TABLE IF EXISTS table_query_3")

# create table for Query 1
user_name = """CREATE TABLE IF NOT EXISTS user_name (song text, userId int, 
                                        firstName text, lastName text, 
                                        PRIMARY KEY (song, userId))"""
# execute the create table
session.execute(user_name)


#==================
# Insert the data
#==================

insert_data_user_name = """INSERT INTO user_name (song, userId, 
                                     firstName, lastName) VALUES (%s, %s, %s, %s)"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # assing csv columns in each variable
        song, userId, = str(line[9]), int(line[10])
        firstName, lastName = str(line[1]), str(line[4])
        
        session.execute(insert_data_user_name, (song, userId, firstName, lastName))
        
#==================
# Validate the data
#==================

# Query 3 in CQL
query_3 = """SELECT firstName, lastName 
                    FROM user_name 
                    WHERE song = %s"""
# ive me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
rows = session.execute(query_3, ('All Hands Against His Own', ))
for row in rows:
    print(f'user first name: {row.firstname:>10},  user last name: {row.lastname}')        

user first name: Jacqueline,  user last name: Lynch
user first name:      Tegan,  user last name: Levine
user first name:       Sara,  user last name: Johnson


## Drop the created tables and close the session and cluster

In [21]:
session.execute("DROP TABLE IF EXISTS song_features")
session.execute("DROP TABLE IF EXISTS artist_song_by_user")
session.execute("DROP TABLE IF EXISTS user_name")                                                

session.shutdown()
cluster.shutdown()

## The End